### 1.0 IMPORTS

In [12]:
import os
import datetime
import pickle

import pandas as pd

pd.options.display.max_columns = None

### 2.0 CONFIGURATIONS

In [13]:
path_raw = './data/'
path_out='./outputs/'
path_out_images='./images/'

model_name="pepsico_frete_ramdom_forest.sav"
name_out=f'RealCaixaPred_{datetime.datetime.now().strftime("%Y-%m-%d %H_%M_%S")}.xlsx'

features= [
    'Planta', 'Movimentação',
    'UF Destino', 'TIPO',
]
eng_classes = [f"{feature}_Class" for feature in features]

### 3.0 LOADING MODEL

In [14]:
model = pickle.load(open(os.path.join(path_out,model_name), 'rb'))

### 4.0 LOADING DATASETS

In [15]:
"""
The input must be formated as the the template file:

Planta  UF Destino  Movimentação    TIPO    KM Ajustado
76956   BA  TRANSFERENCIA   FOODS   580,57
76956   MG  VENDAS  SALTY   1080,45

{
    Planta:Str,
    UF Destino:Str,
    Movimentação:Str,
    TIPO:Str,
    KM Ajustado:Float,
}  

All the possibilities for the categorical features are listed in the files:

Planta_categorical.csv
Movimentação_categorical.csv
UF Destino_categorical.csv
TIPO_categorical.csv

If the user wants to predict the freight cost to a new Source, we recomend to choose an existing source as a proxy to obtain the predictions. 
The model should be retrained to include new features, or include new sources with data avaible.

"""
# INPUT FILE
df_novas_rotas=pd.read_excel(os.path.join(path_raw,'Pepsico_Exemplo_Input_Modelo_Preditivo.xlsx'), sheet_name='Planilha1',dtype={'Planta':'str'})

df_plantas=pd.read_csv(os.path.join(path_out,'Planta_categorical.csv'))
df_movimentacao=pd.read_csv(os.path.join(path_out,'Movimentação_categorical.csv'))
df_uf=pd.read_csv(os.path.join(path_out,'UF Destino_categorical.csv'))
df_tipo=pd.read_csv(os.path.join(path_out,'TIPO_categorical.csv'))

### 5.0 ETL

In [16]:
df_rotas_model = df_novas_rotas.copy()

# Merging categorical classification

df_rotas_model = df_rotas_model.merge(
    df_plantas,
    on=['Planta'],
    how='left'
)

df_rotas_model = df_rotas_model.merge(
    df_uf,
    on=['UF Destino'],
    how='left'
)

df_rotas_model = df_rotas_model.merge(
    df_movimentacao,
    on=['Movimentação'],
    how='left'
)

df_rotas_model = df_rotas_model.merge(
    df_tipo,
    on=['TIPO'],
    how='left'
)

df_rotas_model['VENDAS'] = df_rotas_model['Movimentação'] == 'VENDAS'
df_rotas_model['TRANSFERENCIA'] = df_rotas_model['Movimentação'] == 'TRANSFERENCIA'





### 6.0 PREDICTING

In [17]:
X_novas_rotas = df_rotas_model[['KM Ajustado','VENDAS','TRANSFERENCIA']+eng_classes]

y_novas_rotas= model.predict(X_novas_rotas)

df_rotas_model['RealCaixaPred'] = y_novas_rotas

In [18]:
cols_orig=df_novas_rotas.columns.to_list()

df_rotas_model_out=df_rotas_model[cols_orig+['RealCaixaPred']].copy()

df_rotas_model_out.to_excel(os.path.join(path_out,name_out),index=False)

df_rotas_model_out

,Planta,UF Destino,Movimentação,TIPO,KM Ajustado,RealCaixaPred
0,7962,SP,TRANSFERENCIA,SALTY,722.639,4.763055
1,7962,SP,TRANSFERENCIA,FOODS,722.639,1.334938
2,7956,MG,TRANSFERENCIA,SALTY,1000.000,4.682317
3,7956,BA,VENDAS,FOODS,693.865,7.011714
4,7954,SP,VENDAS,FOODS,330.000,5.810479
5,7954,MT,VENDAS,SALTY,780.000,4.052796
